In [3]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed,MaxPooling1D, Flatten, Conv1D,Conv2D,Dropout, MaxPooling2D # for creating layers inside the Neural Network
from keras.optimizers import Adam , SGD
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import numpy.ma as ma
# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error # for calculating the cost function
from sklearn.ensemble import RandomForestRegressor # for building the model
# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version
from matplotlib import pyplot as plt

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [4]:
df=pd.read_csv('InSAR_data_south/displacement/trainingvalidation.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y-%m-%d')
df=df.dropna(axis=0, how='all')#drop full nan rows
df

,,2014-11-08,2014-12-02,2014-12-26,2015-02-12,2015-03-08,2015-04-01,2015-04-25,2015-05-19,2015-06-12,2015-07-06,...,2018-10-18,2018-10-30,2018-11-11,2018-11-17,2018-11-23,2018-12-05,2018-12-17,2018-12-29,2019-01-10,2019-01-22
Longitude,Latitude,,,,,,,,,,,,,,,,,,,,,
-119.5609,36.1672,0.0,-5.35470,-2.365800,-5.38810,-8.0230,-7.35960,-10.658000,-11.5740,-14.73600,-15.653000,...,-51.2880,-52.2000,-51.5510,-50.8460,-52.1710,-51.1730,-51.0260,-51.1030,-53.1130,-53.6540
-119.5489,36.0072,0.0,-6.98510,-2.425400,-5.59300,-8.2796,-7.10480,-11.352000,-13.1490,-16.28400,-17.528000,...,-80.4520,-81.3120,-81.3700,-81.0500,-82.8020,-81.8950,-80.9560,-82.2640,-83.5630,-83.4070
-119.5369,36.0472,0.0,-6.52240,-2.652600,-7.67760,-11.1110,-10.59000,-14.676000,-17.0200,-20.39000,-22.219000,...,-82.7120,-83.1190,-82.8720,-82.5070,-84.1300,-83.3090,-83.1980,-83.6720,-84.9170,-84.5300
-119.5329,36.0152,0.0,-7.31990,-3.102300,-7.01660,-9.4511,-8.87950,-13.596000,-15.7800,-19.31200,-21.342000,...,-92.7010,-93.6410,-93.4770,-93.2590,-94.9470,-94.3050,-93.6450,-94.7880,-96.0660,-96.0170
-119.5289,35.9512,0.0,-5.69540,-0.297000,-1.17090,-3.5007,-0.99811,-3.422100,-3.9274,-5.57050,-4.514200,...,-31.5990,-31.9420,-31.8580,-31.5220,-32.9980,-32.5920,-31.5850,-32.0960,-33.1620,-32.8450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-118.9351,35.9912,0.0,1.31960,-0.454550,-1.12980,-2.1960,-2.29490,0.732390,-2.6491,-0.57953,-0.463840,...,3.3119,2.4877,3.7906,3.6429,3.8131,2.7015,5.1692,5.8056,3.2722,3.7133
-118.9331,35.8431,0.0,-0.27523,0.340730,0.11561,-1.8977,-1.72400,1.244300,-2.6508,0.44136,0.072182,...,5.3406,4.6145,5.8431,5.6430,5.8181,5.7786,7.0684,8.0765,5.0210,5.4701
-118.9311,35.6111,0.0,-2.60280,0.000017,0.94402,-2.1042,-1.87170,0.510950,-2.7147,0.42639,0.334300,...,4.6175,4.2846,5.3261,5.0863,5.6278,4.1805,6.4902,7.5899,4.7841,5.1107


In [5]:
drain=pd.read_csv('InSAR_data_south/displacement/rainfall.csv')
drain=drain.set_index([drain.columns[0],drain.columns[1]])
drain.columns=pd.to_datetime(drain.columns, format='%Y-%m-%d')
for y in range(df.columns.size):# includes all dates and adds nans
    if df.columns[y] != drain.columns[y]: #if column is misssing insert it
        drain.insert(loc=y,column=df.columns[y],value=np.nan*237,allow_duplicates=False)



C:\Users\jupad\AppData\Local\Temp\ipykernel_1784\2099583455.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  drain.insert(loc=y,column=df.columns[y],value=np.nan*237,allow_duplicates=False)


In [7]:
#Iterate through row until find nan.
#Take date of nan and look for most similar date in following years
dates=drain.columns.values
dates


array(['2014-11-08T00:00:00.000000000', '2014-12-02T00:00:00.000000000',
       '2014-12-26T00:00:00.000000000', '2015-02-12T00:00:00.000000000',
       '2015-03-08T00:00:00.000000000', '2015-04-01T00:00:00.000000000',
       '2015-04-25T00:00:00.000000000', '2015-05-19T00:00:00.000000000',
       '2015-06-12T00:00:00.000000000', '2015-07-06T00:00:00.000000000',
       '2015-07-30T00:00:00.000000000', '2015-08-23T00:00:00.000000000',
       '2015-09-16T00:00:00.000000000', '2015-10-10T00:00:00.000000000',
       '2015-11-03T00:00:00.000000000', '2015-11-27T00:00:00.000000000',
       '2015-12-09T00:00:00.000000000', '2015-12-21T00:00:00.000000000',
       '2016-01-14T00:00:00.000000000', '2016-01-26T00:00:00.000000000',
       '2016-02-07T00:00:00.000000000', '2016-02-19T00:00:00.000000000',
       '2016-03-02T00:00:00.000000000', '2016-03-14T00:00:00.000000000',
       '2016-03-26T00:00:00.000000000', '2016-04-07T00:00:00.000000000',
       '2016-04-19T00:00:00.000000000', '2016-05-01

In [14]:

# for x in drain.index.values:
#     for y in range(len(drain.columns.values)):
#         if np.isnan(drain.loc[x][y]):
#            drain.loc[x][y]= next value
#drain.loc[drain.index.values[0]][109]

59.73682847317471